### Using Model A for our GPU run (best performer)

**Model A** | **Model B** | **Model C**
--- | --- | ---
Max Pooling | Average Pooling | Max Pooling
Same Padding | Same Padding | Valid Padding
97.07% | 93.33% | 96.59%

| **All Models** |
|---|
|INPUT$\to$CONV$\to$POOL$\to$CONV$\to$POOL$\to$FC|
|Convolution Kernel Size = 5$\times$5|
|Convolution Kernel Stride = 1|
|Pooling Kernel Size = 2$\times$2|

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

In [2]:
'''
Step 1: Load the dataset
'''

train_dataset = dsets.MNIST(root="./data",
                            train=True,
                            transform=transforms.ToTensor(), 
                            download=True)

test_dataset = dsets.MNIST(root='./data',
                           train=False,
                           transform=transforms.ToTensor())

'''
Step 2: Make dataset iterable
'''

batch_size = 100
n_iters = 3000
num_epochs = int(n_iters / (len(train_dataset) / batch_size))  #Return as type INT

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

'''
Step 3: Create Model Class (relu)
'''

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        
        # Max Pooling 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        
        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()
        
        # Average Pooling 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
        # Fully Connected Layer
        self.fc1 = nn.Linear(32 * 7 * 7, 10)
        
    def forward(self, x):
        
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)
        
        # Avg Pooling 1
        out = self.maxpool1(out)

        # Convolution 2
        out = self.cnn2(out)
        out = self.relu2(out)
        
        # Avg Pooling 2
        out = self.maxpool2(out)
        
        # Resize for the linear function in the readout layer
        # Original size: (100, 32, 7, 7)
        # out.size(0): 100
        # Desired out size: (100, 32*7*7)
        out = out.view(out.size(0), -1)  #I might need to look into this to see what is going on under the hood
        
        # Linear Function (readout) Layer
        out = self.fc1(out)
        
        return out
'''
Step 4: Instantiate Model Class
'''

model = CNNModel()

#####################
# Use GPU for Model #
#####################

if torch.cuda.is_available():
    model.cuda()

'''
Step 5: Instantiate Loss Class
'''

criterion = nn.CrossEntropyLoss()

'''
Step 6: Instantiate Optimizer Class
'''

learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

'''
Step 7: Train the Model
'''

iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
    
        #####################
        # Use GPU for Model #
        #####################
        if torch.cuda.is_available():
            images = Variable(images.cuda()) #No need to resize, because the image is already in a form that the CNN can use.
            labels = Variable(labels.cuda())
        else:
            images = Variable(images) #No need to resize, because the image is already in a form that the CNN can use.
            labels = Variable(labels)
            
        #Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        #Forward pass to get outputs / logits
        outputs = model(images)
        
        #Calculate Loss: softmax --> Cross Entropy Loss
        loss = criterion(outputs, labels)
        
        #Get gradients w.r.t. parameters
        loss.backward()
        
        #Update parameters
        optimizer.step()
        
        iter += 1
        if iter % 500 == 0:
            # Calculate Accuracy
            correct = 0 
            total = 0
            #Iterate through the test dataset
            for images, labels in test_loader:
                # Load images to Torch Variable
                #####################
                # Use GPU for Model #
                #####################
                if torch.cuda.is_available():
                    images = Variable(images.cuda())
                else:
                    images = Variable(images)
                # Forward pass only to get outputs/logits
                outputs = model(images)
                
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                # Total number of labels
                total += labels.size(0)
              
                # Total correct predictions
                #####################
                # Use GPU for Model #
                #####################
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                else:
                    correct += (predicted == labels).sum()
                    
            accuracy = 100 * correct / total
            
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.data[0], accuracy))

Iteration: 500. Loss: 0.31561365723609924. Accuracy: 90.38
Iteration: 1000. Loss: 0.24880556762218475. Accuracy: 93.57
Iteration: 1500. Loss: 0.23685047030448914. Accuracy: 94.67
Iteration: 2000. Loss: 0.15145353972911835. Accuracy: 95.81
Iteration: 2500. Loss: 0.1142621636390686. Accuracy: 96.53
Iteration: 3000. Loss: 0.1505439728498459. Accuracy: 96.74


## CNN Summary

* Differences from feedforward neural networks
* Convolutional Layers
* Pooling Layers
* Padding
* Ways to expand model's capacity
* Code to convert to running on GPU (see above)
* The same 7 steps apply $\to$ Repetition has been great for retention and I hope, recall. 